In [1]:
from elasticsearch import Elasticsearch
from elasticsearch import helpers
import time
es = Elasticsearch(timeout=30, max_retries=10, retry_on_timeout=True)

In [2]:
new_ettoday = '../data/ettoday/ettoday-new'
BATCH_SIZE = 4000

In [3]:
def getInfo(data):
    start = 0
    for line in data:
        if start is 0:
            start = 1
            continue
        line.encode('UTF-8', 'ignore')
        line = line.strip()
        """
        @

2 @Gais_REC:

3 @url:http://travel.ettoday.net/article/718757.htm

4 @MainTextMD5:892FF0D0A82D5CA1DB70A320611A9BE9

5 @UntagMD5:E9D666D79C1A4D3337720C920F2E5A5F

6 @SiteCode:LvYHeMlIgi

7 @UrlCode:ACC79899BE4BEBDDC236B50C3979023A

8 @title:修杰楷訓練愛女自己吃飯　咘咘完食「拍手鼓掌」萌翻！ | ETtoday 東森旅遊雲 | ETtoday旅遊新聞(影劇)

9 @Size:89230

10 @keyword:東森,記者,藝人,賈靜雯,育兒生活

11 @image_links:http://static.ettoday.net/images/1853/d1853989.jpg

12 @Fetchtime:2017/01/10 23:15:09

13 @post_time:2016/06/17 00:00:00

14 @Ref:http://travel.ettoday.net/article/718839.htm

15 @BodyMD5:818B0989AB17758DA4E17604D1B47052

16 @Lang:utf-8

17 @IP:219.85.79.132

18 @body:
        """
        #print(line)
        if line.startswith('@Gais_REC:'):
            record = dict()
            body = ''
        elif line.startswith('@url:'):
            record['url'] = line[5:]
        elif line.startswith('@MainTextMD5:'):
            record['MainTextMD5'] = line[13:]
        elif line.startswith('@UntagMD5:'):
            record['UntagMD5'] = line[10:]
        elif line.startswith('@SiteCode:'):
            record['SiteCode'] = line[10:]
        elif line.startswith('@UrlCode:'):
            record['UrlCode'] = line[9:]
        elif line.startswith('@title:'):
            record['title'] = line[7:]
        elif line.startswith('@Size:'):
            record['Size'] = line[6:]
        elif line.startswith('@keyword:'):
            record['keyword'] = line[9:].split(',')
        elif line.startswith('@image_links:'):
            record['image_links'] = line[13:]
        elif line.startswith('@Fetchtime:'):
            continue
        elif line.startswith('@Time:'):
            continue
        elif line.startswith('@post_time:'):
            continue
        elif line.startswith('@Ref:'):
            record['Ref'] = line[5:]
        elif line.startswith('@BodyMD5:'):
            record['BodyMD5'] = line[9:]
        elif line.startswith('@Lang:'):
            record['Lang'] = line[6:]
        elif line.startswith('@IP:'):
            record['IP'] = line[4:]
        
        elif line.startswith('@body:'):
            body = body + line[6:]
        elif line.startswith('@'):
            record['content'] = body
            yield record
        else:
            body = body + line
        

In [4]:
cnt = 530279
batchTime = []
batchAvg = 0
batchCnt = 0
start = time.time()
with open(new_ettoday, "r", encoding='UTF-8', errors="surrogateescape") as fileopen:
    data = fileopen

    acts = []
    
    for rec in getInfo(data):

        acts.append({
            '_index': 'news',
            '_type': 'ettoday',
            '_id': cnt,
            '_source': rec,
        })
        cnt += 1
        if len(acts) == BATCH_SIZE:
            batch_start = time.time()
            print('finish {0} datas'.format(cnt))
            helpers.bulk(es, acts)
            batch_end = time.time()
            batch = batch_end - batch_start
            batchTime.append(batch)
            batchAvg += batch;
            batchCnt += 1;
            print("This batch cost {0}s".format(batch))
            acts = []
    if len(acts) > 0:
        helpers.bulk(es, acts)
        acts = []
end = time.time()
print(cnt)
print("total cost {0}s\nAverage batch cost {1}s\n".format((end - start), batchAvg / batchCnt))

finish 534279 datas
This batch cost 16.195317029953003s
finish 538279 datas
This batch cost 11.001287460327148s
finish 542279 datas
This batch cost 11.173154830932617s
finish 546279 datas
This batch cost 9.062031984329224s
finish 550279 datas
This batch cost 11.763473987579346s
finish 554279 datas
This batch cost 11.96469235420227s
finish 558279 datas
This batch cost 11.74701189994812s
finish 562279 datas
This batch cost 10.857375383377075s
finish 566279 datas
This batch cost 7.232305526733398s
finish 570279 datas
This batch cost 11.55960464477539s
finish 574279 datas
This batch cost 11.27107286453247s
finish 578279 datas
This batch cost 15.69124460220337s
finish 582279 datas
This batch cost 11.666321992874146s
finish 586279 datas
This batch cost 7.116475582122803s
finish 590279 datas
This batch cost 9.356776475906372s
finish 594279 datas
This batch cost 10.800898551940918s
finish 598279 datas
This batch cost 9.701912641525269s
finish 602279 datas
This batch cost 8.424516201019287s
fin

This batch cost 9.671610832214355s
finish 1126279 datas
This batch cost 7.794996738433838s
finish 1130279 datas
This batch cost 8.80514669418335s
finish 1134279 datas
This batch cost 8.1956045627594s
finish 1138279 datas
This batch cost 10.277768850326538s
finish 1142279 datas
This batch cost 10.827638149261475s
finish 1146279 datas
This batch cost 10.502944231033325s
finish 1150279 datas
This batch cost 7.896705150604248s
finish 1154279 datas
This batch cost 12.118805408477783s
finish 1158279 datas
This batch cost 8.271049976348877s
finish 1162279 datas
This batch cost 16.696969509124756s
finish 1166279 datas
This batch cost 11.27497410774231s
finish 1170279 datas
This batch cost 15.824040412902832s
finish 1174279 datas
This batch cost 15.03160548210144s
finish 1178279 datas
This batch cost 14.13174557685852s
finish 1182279 datas
This batch cost 13.687344312667847s
finish 1186279 datas
This batch cost 13.046962976455688s
finish 1190279 datas
This batch cost 15.084070444107056s
finish 

finish 1706279 datas
This batch cost 24.175509214401245s
finish 1710279 datas
This batch cost 14.877285957336426s
finish 1714279 datas
This batch cost 16.523360013961792s
finish 1718279 datas
This batch cost 16.104724168777466s
finish 1722279 datas
This batch cost 6.828100681304932s
finish 1726279 datas
This batch cost 9.279344081878662s
finish 1730279 datas
This batch cost 7.9541261196136475s
finish 1734279 datas
This batch cost 9.318058013916016s
finish 1738279 datas
This batch cost 8.92837142944336s
finish 1742279 datas
This batch cost 9.848003149032593s
finish 1746279 datas
This batch cost 10.588404417037964s
finish 1750279 datas
This batch cost 15.211254119873047s
finish 1754279 datas
This batch cost 8.820127964019775s
finish 1758279 datas
This batch cost 10.699041604995728s
finish 1762279 datas
This batch cost 11.399134159088135s
finish 1766279 datas
This batch cost 10.278527975082397s
finish 1770279 datas
This batch cost 9.556572198867798s
finish 1774279 datas
This batch cost 10

This batch cost 15.106231212615967s
finish 2290279 datas
This batch cost 14.187294960021973s
finish 2294279 datas
This batch cost 11.253538131713867s
finish 2298279 datas
This batch cost 13.41582989692688s
finish 2302279 datas
This batch cost 13.318649053573608s
finish 2306279 datas
This batch cost 14.117044448852539s
finish 2310279 datas
This batch cost 13.676386594772339s
finish 2314279 datas
This batch cost 15.136743307113647s
finish 2318279 datas
This batch cost 9.548475503921509s
finish 2322279 datas
This batch cost 10.915897846221924s
finish 2326279 datas
This batch cost 8.844269514083862s
finish 2330279 datas
This batch cost 9.465832710266113s
finish 2334279 datas
This batch cost 8.425660848617554s
finish 2338279 datas
This batch cost 8.776734828948975s
finish 2342279 datas
This batch cost 12.2315514087677s
finish 2346279 datas
This batch cost 10.05691409111023s
finish 2350279 datas
This batch cost 17.920239210128784s
finish 2354279 datas
This batch cost 11.527327299118042s
fini

This batch cost 13.041987895965576s
finish 2870279 datas
This batch cost 10.50676679611206s
finish 2874279 datas
This batch cost 12.23010802268982s
finish 2878279 datas
This batch cost 10.182907342910767s
finish 2882279 datas
This batch cost 13.983131408691406s
finish 2886279 datas
This batch cost 19.152435779571533s
finish 2890279 datas
This batch cost 15.044886350631714s
finish 2894279 datas
This batch cost 22.535953521728516s
finish 2898279 datas
This batch cost 14.047383308410645s
finish 2902279 datas
This batch cost 11.828293800354004s
finish 2906279 datas
This batch cost 13.624027252197266s
finish 2910279 datas
This batch cost 10.725617408752441s
finish 2914279 datas
This batch cost 15.626387357711792s
finish 2918279 datas
This batch cost 13.158902645111084s
finish 2922279 datas
This batch cost 13.015653133392334s
finish 2926279 datas
This batch cost 10.661447048187256s
finish 2930279 datas
This batch cost 8.540465354919434s
finish 2934279 datas
This batch cost 10.712932825088501

This batch cost 12.972741842269897s
finish 3450279 datas
This batch cost 10.297245264053345s
finish 3454279 datas
This batch cost 12.753750085830688s
finish 3458279 datas
This batch cost 21.417398929595947s
finish 3462279 datas
This batch cost 18.73603081703186s
finish 3466279 datas
This batch cost 15.29601788520813s
finish 3470279 datas
This batch cost 12.532414436340332s
finish 3474279 datas
This batch cost 14.762063980102539s
finish 3478279 datas
This batch cost 13.485273599624634s
finish 3482279 datas
This batch cost 10.113930225372314s
finish 3486279 datas
This batch cost 15.766668319702148s
finish 3490279 datas
This batch cost 18.614247798919678s
finish 3494279 datas
This batch cost 9.525682210922241s
finish 3498279 datas
This batch cost 12.7506742477417s
finish 3502279 datas
This batch cost 12.399524450302124s
finish 3506279 datas
This batch cost 10.620973348617554s
finish 3510279 datas
This batch cost 10.836568355560303s
finish 3514279 datas
This batch cost 10.38270902633667s
f

This batch cost 10.926285028457642s
finish 4030279 datas
This batch cost 10.917088508605957s
finish 4034279 datas
This batch cost 14.87705683708191s
finish 4038279 datas
This batch cost 15.55703854560852s
finish 4042279 datas
This batch cost 11.054481744766235s
finish 4046279 datas
This batch cost 8.769538640975952s
finish 4050279 datas
This batch cost 11.64821457862854s
finish 4054279 datas
This batch cost 9.419543981552124s
finish 4058279 datas
This batch cost 10.146601915359497s
finish 4062279 datas
This batch cost 9.677682876586914s
finish 4066279 datas
This batch cost 8.212476253509521s
finish 4070279 datas
This batch cost 8.74418044090271s
finish 4074279 datas
This batch cost 13.220781326293945s
finish 4078279 datas
This batch cost 25.33293867111206s
finish 4082279 datas
This batch cost 9.908078670501709s
finish 4086279 datas
This batch cost 27.277593851089478s
finish 4090279 datas
This batch cost 8.696510553359985s
finish 4094279 datas
This batch cost 14.443217039108276s
finish 

This batch cost 9.913210391998291s
finish 4610279 datas
This batch cost 8.911751985549927s
finish 4614279 datas
This batch cost 11.968019247055054s
finish 4618279 datas
This batch cost 13.463026762008667s
finish 4622279 datas
This batch cost 10.999263525009155s
finish 4626279 datas
This batch cost 9.76181173324585s
finish 4630279 datas
This batch cost 9.516753435134888s
finish 4634279 datas
This batch cost 11.01578950881958s
finish 4638279 datas
This batch cost 9.655372619628906s
finish 4642279 datas
This batch cost 22.650371074676514s
finish 4646279 datas
This batch cost 9.825119495391846s
finish 4650279 datas
This batch cost 9.85163140296936s
finish 4654279 datas
This batch cost 8.744664430618286s
finish 4658279 datas
This batch cost 8.307700872421265s
finish 4662279 datas
This batch cost 8.229803562164307s
finish 4666279 datas
This batch cost 14.936310529708862s
finish 4670279 datas
This batch cost 8.12523365020752s
finish 4674279 datas
This batch cost 12.191600561141968s
finish 467

finish 5190279 datas
This batch cost 10.738282918930054s
finish 5194279 datas
This batch cost 8.336644411087036s
finish 5198279 datas
This batch cost 8.340226411819458s
finish 5202279 datas
This batch cost 11.514453172683716s
finish 5206279 datas
This batch cost 8.611079454421997s
finish 5210279 datas
This batch cost 9.282859563827515s
finish 5214279 datas
This batch cost 14.555559158325195s
finish 5218279 datas
This batch cost 12.57606554031372s
finish 5222279 datas
This batch cost 12.321216106414795s
finish 5226279 datas
This batch cost 11.674352884292603s
finish 5230279 datas
This batch cost 8.803978681564331s
finish 5234279 datas
This batch cost 8.4340078830719s
finish 5238279 datas
This batch cost 10.428205013275146s
finish 5242279 datas
This batch cost 8.793036460876465s
finish 5246279 datas
This batch cost 9.46631383895874s
finish 5250279 datas
This batch cost 9.586475849151611s
finish 5254279 datas
This batch cost 8.169617176055908s
finish 5258279 datas
This batch cost 14.86673

In [5]:
print(batchTime)
print(batchCnt)


[16.195317029953003, 11.001287460327148, 11.173154830932617, 9.062031984329224, 11.763473987579346, 11.96469235420227, 11.74701189994812, 10.857375383377075, 7.232305526733398, 11.55960464477539, 11.27107286453247, 15.69124460220337, 11.666321992874146, 7.116475582122803, 9.356776475906372, 10.800898551940918, 9.701912641525269, 8.424516201019287, 9.84803557395935, 10.530693769454956, 12.320157051086426, 11.290719985961914, 20.97006320953369, 17.11069107055664, 10.404229879379272, 16.45855736732483, 22.894152402877808, 15.459808588027954, 13.146328210830688, 13.518831729888916, 12.733680248260498, 10.786097288131714, 9.240229606628418, 7.8282599449157715, 10.21218752861023, 8.177286386489868, 9.770758152008057, 10.046002626419067, 9.255612134933472, 9.529495000839233, 11.847274541854858, 12.86655879020691, 13.147900581359863, 11.11751127243042, 18.805131196975708, 17.83625364303589, 18.564952850341797, 16.03847599029541, 20.85687565803528, 23.306335926055908, 18.909606218338013, 10.338

In [1]:
for i in batchTime:
    print(i)

NameError: name 'batchTime' is not defined